# Analyzing GRB 080916C
![Alt text](https://astrobites.org/wp-content/uploads/2014/10/NASAGRBwhoa-1024x576.jpg)
*(NASA/Swift/Cruz deWilde)*

To demonstrate the capabilities and features of 3ML in, we will go through a time-integrated and time-resolved analysis. This example serves as a standard way to analyze Fermi-GBM data with 3ML as well as a template for how you can design your instrument's analysis pipeline with 3ML if you have similar data.

3ML provides utilities to reduce time series data to plugins in a *correct* and *statistically justified* way (e.g., background fitting of Poisson data is done with a Poisson likelihood). The approach is generic and can be extended. For more details, see the [time series documentation](https://threeml.readthedocs.io/en/stable/notebooks/Building_Plugins_from_TimeSeries.html).

In [ ]:
import warnings
warnings.simplefilter("ignore")



In [ ]:
%%capture
import matplotlib.pyplot as plt
import numpy as np
np.seterr(all="ignore")


from threeML import *
from threeML.io.package_data import get_path_of_data_file

In [ ]:

silence_warnings()
%matplotlib inline
from jupyterthemes import jtplot
jtplot.style(context="talk", fscale=1, ticks=True, grid=False)
set_threeML_style()


## Examining the catalog

As with Swift and Fermi-LAT, 3ML provides a simple interface to the on-line Fermi-GBM catalog. Let's get the information for GRB 080916C.

In [ ]:
gbm_catalog = FermiGBMBurstCatalog()
gbm_catalog.query_sources("GRB080916009")

To aid in quickly replicating the catalog analysis, and thanks to the tireless efforts of the Fermi-GBM team, we have added the ability to extract the analysis parameters from the catalog as well as build an **astromodels** model with the best fit parameters baked in. Using this information, one can quickly run through the catalog an replicate the entire analysis with a script. Let's give it a try.

In [ ]:
grb_info =  gbm_catalog.get_detector_information()["GRB080916009"]

gbm_detectors = grb_info['detectors']
source_interval = grb_info["source"]["fluence"]
background_interval = grb_info["background"]["full"]
best_fit_model = grb_info["best fit model"]["fluence"]
model =  gbm_catalog.get_model(best_fit_model, "fluence")["GRB080916009"]

In [ ]:
model

## Downloading the data
We provide a simple interface to download the Fermi-GBM data. Using the information from the catalog that we have extracted, we can download just the data from the detectors that were used for the catalog analysis. This will download the CSPEC, TTE and instrument response files from the on-line database. 



In [ ]:
dload = download_GBM_trigger_data("bn080916009",detectors=gbm_detectors)

Let's first examine the catalog fluence fit. Using the **TimeSeriesBuilder**, we can fit the background, set the source interval, and create a 3ML plugin for the analysis. We will loop through the detectors, set their appropriate channel selections, and ensure there are enough counts in each bin to make the [PGStat profile likelihood valid](https://giacomov.github.io/Bias-in-profile-poisson-likelihood/).

* First we use the CSPEC data to fit the background using the background selections. We use CSPEC because it has a longer duration for fitting the background. 
* The background is saved to an HDF5 file that stores the polynomial coefficients and selections which we can read in to the TTE file later.
* The light curve is plotted.
* The source selection from the catalog is set and **DispersionSpectrumLike** plugin is created.
* The plugin has the standard GBM channel selections for spectral analysis set.

In [ ]:
fluence_plugins = []
time_series = {}
for det in gbm_detectors:

    
    
    ts_cspec = TimeSeriesBuilder.from_gbm_cspec_or_ctime(
    det, cspec_or_ctime_file=dload[det]["cspec"], rsp_file=dload[det]["rsp"]
    )

    ts_cspec.set_background_interval(*background_interval.split(","))
    ts_cspec.save_background(f"{det}_bkg.h5", overwrite=True)

    ts_tte = TimeSeriesBuilder.from_gbm_tte(
        det,
        tte_file=dload[det]["tte"],
        rsp_file=dload[det]["rsp"],
        restore_background=f"{det}_bkg.h5",
    )
    
    time_series[det] = ts_tte

    ts_tte.set_active_time_interval(source_interval)

    ts_tte.view_lightcurve(-40, 100)
    
    fluence_plugin = ts_tte.to_spectrumlike()
    
    if det.startswith("b"):
        
        fluence_plugin.set_active_measurements("250-30000")
    
    else:
        
        fluence_plugin.set_active_measurements("9-900")
    
    fluence_plugin.rebin_on_background(1.)
    
    fluence_plugins.append(fluence_plugin)

## Setting up the fit
Let's see if we can reproduce the results from the catalog. 

### Set priors for the model
We will fit the spectrum using Bayesian analysis, so we must set priors on the model parameters. 


In [ ]:
model.GRB080916009.spectrum.main.shape.alpha.prior = Truncated_gaussian(lower_bound = -1.5, upper_bound = 1, mu=-1, sigma=0.5)
model.GRB080916009.spectrum.main.shape.beta.prior = Truncated_gaussian(lower_bound = -5, upper_bound = -1.6, mu=-2.25, sigma=0.5)
model.GRB080916009.spectrum.main.shape.break_energy.prior = Log_normal(mu=2, sigma=1)
model.GRB080916009.spectrum.main.shape.break_energy.bounds = (None, None)
model.GRB080916009.spectrum.main.shape.K.prior = Log_uniform_prior(lower_bound = 1E-3, upper_bound = 1E1)
model.GRB080916009.spectrum.main.shape.break_scale.prior = Log_uniform_prior(lower_bound = 1E-4, upper_bound = 10)

### Clone the model and setup the Bayesian analysis class
Next, we clone the model we built from the catalog so that we can look at the results later and fit the cloned model. We pass this model and the **DataList** of the plugins to a **BayesianAnalysis** class and set the sampler to MultiNest. 

In [ ]:
new_model = clone_model(model)

bayes = BayesianAnalysis(new_model, DataList(*fluence_plugins))

# share spectrum gives a linear speed up when
# spectrumlike plugins have the same RSP input energies
bayes.set_sampler("multinest", share_spectrum=True)


### Examine at the catalog fitted model
We can quickly examine how well the catalog fit matches the data. There appears to be a discrepancy between the data and the model! Let's refit to see if we can fix it.  

In [ ]:
fig = display_spectrum_model_counts(bayes, min_rate=20, step=False );

### Run the sampler
We let MultiNest condition the model on the data

In [ ]:
bayes.sampler.setup(n_live_points=400)
bayes.sample()

Now our model seems to match much better with the data! 

In [ ]:
bayes.restore_median_fit()
fig = display_spectrum_model_counts(bayes, min_rate=20);

But how different are we from the catalog model? Let's plot our fit along with the catalog model. Luckily, 3ML can handle all the units for is

In [ ]:
conversion = u.Unit('keV2/(cm2 s keV)').to('erg2/(cm2 s keV)')
energy_grid = np.logspace(1,4, 100)*u.keV
vFv = (energy_grid**2* model.get_point_source_fluxes(0, energy_grid)).to('erg2/(cm2 s keV)')



In [ ]:
fig = plot_spectra(bayes.results, flux_unit='erg2/(cm2 s keV)');
ax = fig.get_axes()[0]
_ = ax.loglog(energy_grid, vFv, color='blue', label='catalog model')

## Time Resolved Analysis 

Now that we have examined fluence fit, we can move to performing a time-resolved analysis.

### Selecting a temporal binning

We first get the brightest NaI detector and create time bins via the Bayesian blocks algorithm. We can use the fitted background to make sure that our intervals are chosen in an unbiased way.

In [ ]:
n3 = time_series['n3']

In [ ]:
n3.create_time_bins(0,60, method="bayesblocks", use_background=True, p0=0.2)

Sometimes, glitches in the GBM data cause spikes in the data that the Bayesian blocks algorithm detects as fast changes in the count rate. We will have to remove those intervals manually.

<div class="alert alert-info">

**Note:** In the future, 3ML will provide an automated method to remove these unwanted spikes.

</div>


In [ ]:
fig = n3.view_lightcurve(use_binner=True);

In [ ]:
bad_bins = []
for i, w in enumerate(n3.bins.widths):
    
    if w < 5E-2:
        bad_bins.append(i)
    
    
edges = [n3.bins.starts[0]]

for i,b in enumerate(n3.bins):
    
    if i not in bad_bins:        
        edges.append(b.stop)

starts=edges[:-1]
stops = edges[1:]


n3.create_time_bins(starts, stops, method='custom')

Now our light curve looks much more acceptable.

In [ ]:
fig = n3.view_lightcurve(use_binner=True);

The time series objects can read time bins from each other, so we will map these time bins onto the other detectors' time series and create a list of time plugins for each detector and each time bin created above. 

In [ ]:
time_resolved_plugins = {}

for k,v in time_series.items():
    v.read_bins(n3)
    time_resolved_plugins[k] = v.to_spectrumlike(from_bins=True)

### Setting up the model
For the time-resolved analysis, we will fit the classic *Band* function to the data. We will set some principled priors.

In [ ]:
band = Band()
band.alpha.prior = Truncated_gaussian(lower_bound = -1.5, upper_bound = 1, mu=-1, sigma=0.5)
band.beta.prior = Truncated_gaussian(lower_bound = -5, upper_bound = -1.6, mu=-2, sigma=0.5)
band.xp.prior = Log_normal(mu=2, sigma=1)
band.xp.bounds = (0, None)
band.K.prior = Log_uniform_prior(lower_bound=1E-10, upper_bound=1E3)
ps = PointSource('grb', 0,0, spectral_shape=band)
band_model = Model(ps)



### Perform the fits

One way to perform Bayesian spectral fits to all the intervals is to loop through each one. There can are many ways to do this, so find an analysis pattern that works for you.



In [ ]:
models = []
results = []
analysis = []
for interval in range(12):

    # clone the model above so that we have a separate model
    # for each fit

    this_model = clone_model(band_model)

    # for each detector set up the plugin
    # for this time interval

    this_data_list = []
    for k, v in time_resolved_plugins.items():

        pi = v[interval]

        if k.startswith("b"):
            pi.set_active_measurements("250-30000")
        else:
            pi.set_active_measurements("9-900")

        pi.rebin_on_background(1.0)

        this_data_list.append(pi)

    # create a data list

    dlist = DataList(*this_data_list)

    # set up the sampler and fit

    bayes = BayesianAnalysis(this_model, dlist)
    
    # get some speed with share spectrum
    bayes.set_sampler("multinest", share_spectrum=True) 
    bayes.sampler.setup(n_live_points=500)
    bayes.sample()

    # at this stage we coudl also
    # save the analysis result to
    # disk but we will simply hold
    # onto them in memory

    analysis.append(bayes)

### Examine the fits
Now we can look at the fits in count space to make sure they are ok.

In [ ]:
for a in analysis:
    a.restore_median_fit()
    _ = display_spectrum_model_counts(a, min_rate=[20,20,-99], step=False)

Finally, we can plot the models together to see how the spectra evolve with time.

In [ ]:
fig = plot_spectra(*[a.results for a in analysis[::1]], flux_unit="erg2/(cm2 s keV)", fit_cmap='viridis', contour_cmap='viridis', contour_style_kwargs=dict(alpha=0.1));

This example can serve as a template for performing analysis on GBM data. However, as 3ML provides an abstract interface and modular building blocks, similar analysis pipelines can be built for any time series data. 